In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#IMPORTING THE DATASET, SHOWING THE COLUMNS
df= pd.read_csv("/kaggle/input/let-me-introduce-myself/final_youtube_history_data.csv")
print(df.columns)

with open("/kaggle/input/let-me-introduce-myself/all_tags_count.txt","r",encoding="utf-8")as f:
    tags_raw_data=f.read()
    tags_raw_data=tags_raw_data

#IMPORTANT DATA
RECORD_START_YEAR=df["SEEN_YEAR"].min()
RECORD_END_YEAR=(df["SEEN_YEAR"].max())+1
CATEGORIES_DICT={1:"Film & Animation",2:"Autos & Vehicles",10:"Music",15:"Pets & Animals",17:"Sports",19:"Travel & Events",20:"Gaming",22:"People & Blogs",23:"Comedy",24:"Entertainment",25:"News & Politics",26:"Howto & Style",27:"Education",28:"Science & Technology",29:"Nonprofits & Activism",44:"Trailer"}



In [ ]:
#FINDING MOST PLAYED CHANNELS EVER
top_20_channels=df[['CHANNEL']].groupby('CHANNEL').size().sort_values(ascending=False).head(20)

top_20_channels.plot.pie(figsize=(6, 10), title="TOP 20 MOST PLAYED CHANNELS")
#print(top_20_channels)

times_played_top_20=top_20_channels.sum()
times_played_channels=(df[["CHANNEL"]].groupby("CHANNEL").size().sum())-times_played_top_20

top_20_vs_global=pd.DataFrame({"COUNT":[times_played_top_20,times_played_channels]},index=["TOP 20 CHANNELS","OTHERS"])
top_20_vs_global.plot.pie(y='COUNT', figsize=(5, 5), autopct='%1.1f%%')


In [ ]:
#FINDING MOST PLAYED VIDEOS EVER
top_20_videos=df[['TITLE']].groupby('TITLE').size().sort_values(ascending=False).head(20)
print(top_20_videos)
top_20_videos.plot.pie(y="TIMES", figsize=(6, 10), title="TOP 20 MOST PLAYED VIDEOS");

In [ ]:
#MONTHLY MOST CLICKED CHANNEL

#SORTING DATAFRAME BY SEEN_YEAR AND SEEN_MONTH
a=df[['SEEN_YEAR','SEEN_MONTH','CHANNEL']].sort_values(['SEEN_YEAR','SEEN_MONTH'])

#ADDING THE "COUNT" COLUMN
b=a.groupby(['SEEN_YEAR',"SEEN_MONTH","CHANNEL"])['CHANNEL'].count().reset_index(name="COUNT")

temp_data=[]

for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
    values_per_year=(b[b["SEEN_YEAR"]==int(year)])
    for month in range (1,12):
        #EVERYTHING SEGMENTED BY YEAR + MONTH
        max_amount_views=values_per_year[values_per_year["SEEN_MONTH"]==int(month)]
        
        #FILTERING THE MAX COUNT-VALUE FOR EVERY MONTH.
        filter_1=max_amount_views["COUNT"].max()
        
        #SKIPPING EVERY nan VALUE (MONTHS WHERE THERE IS NO RECORD)
        if type(filter_1)!=float:
            filtered_data=(max_amount_views[max_amount_views["COUNT"]==filter_1]).values.tolist()
            #EN FILTERED_DATA ES DONDE PUEDEN EXISTIR MÁS DE UN MÁXIMO! EN LA SIGUIENTE LÍNEA SE ESTA
            #AGREGANDO ÚNICAMENTE EL PRIMER VALOR(LISTA) DE LA TUPLA 
            temp_data.append(filtered_data[0])

#CREATING THE FINAL DATAFRAME            
final=pd.DataFrame(temp_data, columns=['SEEN_YEAR','SEEN_MONTH',"TITLE","COUNT"])
final["TOP CHANNELS PER MONTH"]="("+final["SEEN_YEAR"].astype(str)+"-"+final["SEEN_MONTH"].astype(str)+")  --  "+final["TITLE"]
final.plot.bar(y="COUNT", x="TOP CHANNELS PER MONTH", figsize =(18,6), title="MOST SEEN CHANNELS PER MONTH")



In [ ]:
#MONTHLY MOST CLICKED VIDEOS

#SORTING DATAFRAME BY SEEN_YEAR AND SEEN_MONTH
a=df[['SEEN_YEAR','SEEN_MONTH','TITLE']].sort_values(['SEEN_YEAR','SEEN_MONTH'])

#ADDING THE "COUNT" COLUMN
b=a.groupby(['SEEN_YEAR',"SEEN_MONTH","TITLE"])['TITLE'].count().reset_index(name="COUNT")

temp_data=[]

for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
    values_per_year=(b[b["SEEN_YEAR"]==int(year)])
    for month in range (1,12):
        #EVERYTHING SEGMENTED BY YEAR + MONTH
        max_amount_views=values_per_year[values_per_year["SEEN_MONTH"]==int(month)]
        
        #FILTERING THE MAX COUNT-VALUE FOR EVERY MONTH.
        filter_1=max_amount_views["COUNT"].max()
        
        #SKIPPING EVERY nan VALUE (MONTHS WHERE THERE IS NO RECORD)
        if type(filter_1)!=float:
            filtered_data=(max_amount_views[max_amount_views["COUNT"]==filter_1]).values.tolist()
            #EN FILTERED_DATA ES DONDE PUEDEN EXISTIR MÁS DE UN MÁXIMO! EN LA SIGUIENTE LÍNEA SE ESTA
            #AGREGANDO ÚNICAMENTE EL PRIMER VALOR(LISTA) DE LA TUPLA 
            temp_data.append(filtered_data[0])

#CREATING THE FINAL DATAFRAME            
final=pd.DataFrame(temp_data, columns=['SEEN_YEAR','SEEN_MONTH',"TITLE","COUNT"])
final["TOP VIDEOS PER MONTH"]="("+final["SEEN_YEAR"].astype(str)+"-"+final["SEEN_MONTH"].astype(str)+")  --  "+final["TITLE"].sort_values(ascending=True)

final.plot.barh(y="COUNT", x="TOP VIDEOS PER MONTH", figsize =(10,18), title="MOST SEEN VIDEOS PER MONTH")

In [ ]:
#HISTORICAL DAILY CONSUMPTION SEGMENTED BY HOUR.

data=df[["SEEN_YEAR","SEEN_MONTH"]]
data["TIME"]=df["SEEN_TIME"].str.slice(stop=2).replace(':','',regex=True)

temp_final=[]

for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
    #SELECTS VALUES PER YEAR
    values_per_year=(data[data["SEEN_YEAR"]==int(year)])
    for month in range (1,12):
        #SELECTS VALUES PER MONTH
        values_per_month=values_per_year[values_per_year["SEEN_MONTH"]==int(month)]
        #GROUPS EVERYTHING AND ADDS THE COUNT COLUMN
        grouped=values_per_month.groupby(["SEEN_YEAR","SEEN_MONTH","TIME"])["TIME"].count().reset_index(name="COUNT")
        #APPENDS THE GROUPED DATA INTO "temp_final"
        temp_final.append(grouped)

#CREATING THE BASE-DATAFRAME
f=pd.DataFrame(temp_final[0])
#ADDING EVERY OBSERVATION 
final=f.append(temp_final[1:])
final["DATA"]=final["SEEN_YEAR"].astype(str)+" "+final["SEEN_MONTH"].astype(str)+" "+final["TIME"].astype(str)
final.plot.bar(x="DATA", y="COUNT", figsize =(250,10))

    

In [ ]:
#TOP 50 MUSIC VIDEOS.

MUSIC_CATEGORY_ID=10

top_50_raw_data=df[["TITLE"]]
top_50=(top_50_raw_data[df["VIDEO_CATEGORY"]==MUSIC_CATEGORY_ID]).groupby("TITLE").size().sort_values(ascending=False).head(50)


#top_50.plot.pie( figsize=(6, 15), title="TOP 50 MUSIC SONGS")

print(top_50)

In [ ]:
#TOP MUSIC VIDEO PER MONTH

MUSIC_CATEGORY_ID=10

#SORTING DATAFRAME BY SEEN_YEAR AND SEEN_MONTH
a=df[['SEEN_YEAR','SEEN_MONTH','TITLE']].sort_values(['SEEN_YEAR','SEEN_MONTH'])

b=(a[df["VIDEO_CATEGORY"]==MUSIC_CATEGORY_ID])
#ADDING THE "COUNT" COLUMN
c=b.groupby(['SEEN_YEAR',"SEEN_MONTH","TITLE"])['TITLE'].count().reset_index(name="COUNT")

temp_data=[]

for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
    values_per_year=(c[c["SEEN_YEAR"]==int(year)])
    for month in range (1,12):
        #EVERYTHING SEGMENTED BY YEAR + MONTH
        max_amount_views=values_per_year[values_per_year["SEEN_MONTH"]==int(month)]
        
        #FILTERING THE MAX COUNT-VALUE FOR EVERY MONTH.
        filter_1=max_amount_views["COUNT"].max()
        
        #SKIPPING EVERY nan VALUE (MONTHS WHERE THERE IS NO RECORD)
        if type(filter_1)!=float:
            filtered_data=(max_amount_views[max_amount_views["COUNT"]==filter_1]).values.tolist()
            #EN FILTERED_DATA ES DONDE PUEDEN EXISTIR MÁS DE UN MÁXIMO! EN LA SIGUIENTE LÍNEA SE ESTA
            #AGREGANDO ÚNICAMENTE EL PRIMER VALOR(LISTA) DE LA TUPLA 
            temp_data.append(filtered_data[0])
            
#CREATING THE FINAL DATAFRAME            
final=pd.DataFrame(temp_data, columns=['SEEN_YEAR','SEEN_MONTH',"TITLE","COUNT"])
final["TOP SONG PER MONTH"]="("+final["SEEN_YEAR"].astype(str)+"-"+final["SEEN_MONTH"].astype(str)+")  --  "+final["TITLE"]
final.plot.bar(y="COUNT", x="TOP SONG PER MONTH", figsize =(18,6), title="MOST PLAYED SONGS PER MONTH")

In [ ]:
#TIEMPO PROMEDIO POR CATEGORIA HISTÓRICO

In [ ]:
#TIEMPO PROMEDIO POR CATEGORIA - MENSUAL

In [ ]:
#SIN TERMINAR
#TIEMPO PROMEDIO POR MES vs. INTERES


LANGUAGES_TIME_RAW=df[["DESCRIPTION_DETECTED_LANG","SEEN_YEAR","SEEN_MONTH","VIDEO_DURATION"]]
LANGUAGES_TIME_RAW=LANGUAGES_TIME_RAW.groupby(["SEEN_YEAR","SEEN_MONTH","DESCRIPTION_DETECTED_LANG"]).count()
#LANGUAGES_TIME_RAW.plot.bar(stacked=True, figsize=(150,10))
print(LANGUAGES_TIME_RAW)


In [ ]:
#TIEMPO POR IDIOMA Y MENSUAL

In [ ]:
#HISTORICO IDIOMAS
LANGUAGES_GLOBAL=df[["DESCRIPTION_DETECTED_LANG"]].groupby(["DESCRIPTION_DETECTED_LANG"]).size().reset_index(name="COUNT")

#Sort the DataFrame in descending order; will create a Series
RAW_LANGUAGES_GLOBAL = LANGUAGES_GLOBAL.sort_values(by="COUNT",ascending=False)

#LENGTH OF THE DATAFRAME
len_RAW_LANGUAGES_GLOBAL=(len(RAW_LANGUAGES_GLOBAL))
#SELECTING [:4]
top=RAW_LANGUAGES_GLOBAL.head(4).reset_index()
#SELECTING [4:]
tail_RAW_LANGUAGES_GLOBAL=len_RAW_LANGUAGES_GLOBAL-4
tail=RAW_LANGUAGES_GLOBAL.tail(tail_RAW_LANGUAGES_GLOBAL).sum()

f=top.append({"DESCRIPTION_DETECTED_LANG":"OTHER","COUNT":tail[1]}, ignore_index=True)
f=f[["DESCRIPTION_DETECTED_LANG","COUNT"]].set_index("DESCRIPTION_DETECTED_LANG")
print(f)
f.plot.pie(y="COUNT", figsize=(7,7), autopct='%1.1f%%')



In [ ]:
#HISTORICO IDIOMAS POR CATEGORIA

LANGUAGES_GLOBAL=df[["DESCRIPTION_DETECTED_LANG","VIDEO_CATEGORY"]].replace(CATEGORIES_DICT)
LANGUAGES_GLOBAL=LANGUAGES_GLOBAL.groupby(["VIDEO_CATEGORY","DESCRIPTION_DETECTED_LANG"]).size().reset_index(name="COUNT").sort_values(by="COUNT",ascending=False)

for category in CATEGORIES_DICT.values():
    #FILTERING BY CATEGORIES
    filtered_language_global=(LANGUAGES_GLOBAL[LANGUAGES_GLOBAL["VIDEO_CATEGORY"]==category]).set_index("DESCRIPTION_DETECTED_LANG")

    #LENGTH OF THE DATAFRAME
    len_filtered_language_global=(len(filtered_language_global))
    #SELECTING [:4]
    top=filtered_language_global.head(4).reset_index()
    #SELECTING [4:]
    tail_filtered_language_global=len_filtered_language_global-4
    tail=filtered_language_global.tail(tail_filtered_language_global).sum()

    f=top.append({"DESCRIPTION_DETECTED_LANG":"OTHER","COUNT":tail[1]}, ignore_index=True)
    f=f[["DESCRIPTION_DETECTED_LANG","COUNT"]].set_index("DESCRIPTION_DETECTED_LANG")
    f.plot.pie(title=category, y="COUNT", figsize=(5,5), autopct='%1.1f%%')

In [ ]:
#HISTORICAL CATEGORIES 
global_categories_raw=df[["VIDEO_CATEGORY"]].replace(CATEGORIES_DICT).groupby("VIDEO_CATEGORY").size()
global_categories_raw.plot.pie(figsize=(8,8), autopct='%1.1f%%')

In [ ]:
#LANGUAGE ALONG TIME

languages_along_time=df[["DESCRIPTION_DETECTED_LANG","SEEN_YEAR","SEEN_MONTH"]]

def Languages_Along_Time(languages_along_time):
    temp_data=[]

    for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
        values_per_year=(languages_along_time[languages_along_time["SEEN_YEAR"]==int(year)])
        for month in range (1,12):
            #EVERYTHING SEGMENTED BY YEAR + MONTH
            lang_per_month=values_per_year[values_per_year["SEEN_MONTH"]==int(month)].groupby(["SEEN_YEAR","SEEN_MONTH","DESCRIPTION_DETECTED_LANG"]).size().reset_index(name="COUNT").sort_values(by="COUNT",ascending=False)
            #LENGTH OF THE DATAFRAME
            len_lang_per_month=(len(lang_per_month))
            #SELECTING [:4]
            top=lang_per_month.head(4).reset_index()
            #SELECTING [4:]
            tail_lang_per_month=len_lang_per_month-4
            tail=lang_per_month.tail(tail_filtered_language_global).sum()
            
            #APPENDING EVERY SINGLE TAIL VALUE TO THE TOP df
            f=top.append({"SEEN_YEAR":year,"SEEN_MONTH":month,"DESCRIPTION_DETECTED_LANG":"oth","COUNT":tail[1]}, ignore_index=True)
            #CREATING DATE COLUMN WHICH WILL SIMPLIFY THE PLOTTING
            f["DATE"]=(f["SEEN_YEAR"].astype(str)+"-"+f["SEEN_MONTH"].astype(str))
            #CONVERTING THE RESULT AS A LIST
            f=f[["DATE","DESCRIPTION_DETECTED_LANG","COUNT"]].values.tolist()
            
            for value in f:
                #EXPORTING RESULTS TO TEMP_DATA
                temp_data.append(value)
    #RETURNS TEMP_DATA AS A LIST WITH THE RESULTS.
    return temp_data

temp_data=Languages_Along_Time(languages_along_time)
final=pd.DataFrame(temp_data, columns=["DATE","DESCRIPTION_DETECTED_LANG","COUNT"])
final.plot.bar(x="DESCRIPTION_DETECTED_LANG",y="COUNT",figsize=(150,8),stacked=True)


In [ ]:
#LANGUAGES PER YEAR
#SIN TERMINAR

languages_along_time=df[["DESCRIPTION_DETECTED_LANG","SEEN_YEAR"]]

def Languages_Along_Time(languages_along_time):
    temp_data=[]

    for year in range(RECORD_START_YEAR,RECORD_END_YEAR):
        values_per_year=(languages_along_time[languages_along_time["SEEN_YEAR"]==int(year)])
        values_per_year=values_per_year.groupby(["SEEN_YEAR","DESCRIPTION_DETECTED_LANG"]).size().reset_index(name="COUNT").sort_values("COUNT",ascending=False)
        len_values_per_year=(len(values_per_year))
        #SELECTING [:4]
        top=values_per_year.head(4).reset_index()
        #SELECTING [4:]
        tail_values_per_year=len_values_per_year-4
        tail=values_per_year.tail(tail_values_per_year).sum()

        #APPENDING EVERY SINGLE TAIL VALUE TO THE TOP df
        f=top.append({"SEEN_YEAR":year,"DESCRIPTION_DETECTED_LANG":"oth","COUNT":tail[2]}, ignore_index=True).set_index("DESCRIPTION_DETECTED_LANG")
        #CREATING DATE COLUMN WHICH WILL SIMPLIFY THE PLOTTING
        f.plot.pie(y="COUNT",title=year)
        
        
            
Languages_Along_Time(languages_along_time)

In [ ]:
#RESTRICTED CONTENT vs. CATEGORIES.

a=df[["VIDEO_CATEGORY"]].replace(CATEGORIES_DICT)
b=(a[df["VIDEO_RATING"]!="{}"]).groupby("VIDEO_CATEGORY").size()

b.plot.pie(figsize=(10, 10), title="RESTRICTED CONTENT SEGMENTED BY CATEGORIES", autopct='%1.1f%%')
